Universidade Federal de Alagoas

IC - Instituto de Computação

 

# Processamento de linguagem natural - 2020.1
**Professor**: Thales Vieira

**Alunos**: Hugo Tallys Martins Oliveira e Valério Nogueira Rodrigues Júnior


## 6ª lista de exercícios

---

### Pré-processamento dos dados

In [38]:
import nltk
import numpy
import pandas
import random
import re
import time
import umap

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20
from bokeh.plotting import figure
from gensim.models import KeyedVectors
from IPython.display import HTML, display
from keras.layers import Conv1D, Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF, PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from yellowbrick.cluster import KElbowVisualizer

In [2]:
output_notebook() # Necessário para visualizar os gráficos com bokeh

Loading BokehJS ...

In [3]:
nltk.download('stopwords'); nltk.download('rslp'); nltk.download('punkt');

[nltk_data] Downloading package stopwords to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
bbc_dataset_url = 'data/bbc.csv'
cnn_dataset_url = 'data/cnn.csv'

bbc = pandas.read_csv(bbc_dataset_url, sep='|')
bbc['label'] = 'BBC'
cnn = pandas.read_csv(cnn_dataset_url, sep='|')
cnn['label'] = 'CNN'

dataset = pandas.concat([bbc, cnn], ignore_index=True)
dataset = dataset.dropna(axis=0).reset_index(drop=True)

In [5]:
dataset.text = dataset.text.apply(lambda text: text.replace('\n', ' ')) # Remoção das quebras de linha
dataset.title = dataset.title.apply(lambda text: text.replace('\n', ' '))

In [6]:
def remove_boilerplate(text):
    boilerplate = ['Compartilhe este post com Email Facebook Messenger Messenger Twitter WhatsApp LinkedIn Copiar este link Estes são links externos e abrirão numa nova janela', 'Já assistiu aos nossos novos vídeos no YouTube? Inscreva-se no nosso canal!', 'Final de YouTube post  de BBC News Brasil Final de YouTube post 2 de BBC News Brasil Final de YouTube post 3 de BBC News Brasil']
    
    for b in boilerplate:
        text = text.replace(b, '')
    return text

dataset.text = dataset.text.apply(remove_boilerplate) # Remoção de fragmentos irrelevantes do texto que se repetem

In [7]:
def preprocess(text):
    text = re.sub(r'\w*\d\w*', '', text) # Remove todas as palavras que contém números
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', text.lower()) # Remove pontuação e converte para minúscula
    return re.sub(r'\s+', ' ', text) # Remove espaços repetidos

dataset['processed_text'] = dataset.text.apply(preprocess)
dataset['processed_title'] = dataset.title.apply(preprocess)

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def tokenize_remove_stopwords(text):
    tokenized_text = nltk.word_tokenize(text, language='portuguese')
    return " ".join([token for token in tokenized_text if token not in stopwords])

dataset.processed_text = dataset.processed_text.apply(tokenize_remove_stopwords) # Tokeniza o texto e remove stopwords
dataset.processed_title = dataset.processed_title.apply(tokenize_remove_stopwords) # Tokeniza o titulo e remove stopwords

In [9]:
dataset

,url,title,text,label,processed_text,processed_title
0,https://www.bbc.co.uk/portuguese/brasil-53020785,Coronavírus: pandemia pode jogar até 14 milhõe...,A turbulência econômica causada pela pandemi...,BBC,turbulência econômica causada pandemia novo co...,coronavírus pandemia pode jogar milhões brasil...
1,https://www.bbc.co.uk/portuguese/brasil-53027318,Coronavírus: como funcionam as duas vacinas co...,Cerca de 11 mil voluntários brasileiros vão ...,BBC,cerca mil voluntários brasileiros vão receber ...,coronavírus funcionam duas vacinas contra covi...
2,https://www.bbc.co.uk/portuguese/brasil-51713943,Coronavírus: Brasil passa o Reino Unido e se t...,*atualizada às 18h20 de 12 de junho de 2020 ...,BBC,atualizada s junho brasil totalizou nesta sext...,coronavírus brasil passa reino unido torna seg...
3,https://www.bbc.co.uk/portuguese/internacional...,Coronavírus na Índia: com lockdown 'insustentá...,"Quando, em 24 de março, o governo indiano in...",BBC,março governo indiano iniciou estrito isolamen...,coronavírus índia lockdown insustentável índia...
4,https://www.bbc.co.uk/portuguese/internacional...,2ª onda do coronavírus? Irã vê aumento acelera...,O Irã registrou um rápido aumento no número ...,BBC,irã registrou rápido aumento número casos covi...,onda coronavírus irã vê aumento acelerado após...
...,...,...,...,...,...,...
1611,https://www.cnnbrasil.com.br/saude/2020/02/27/...,Farmácias têm falta de máscaras após confirmaç...,Com a confirmação do primeiro caso de contamin...,CNN,confirmação primeiro caso contaminação novo co...,farmácias têm falta máscaras após confirmação ...
1612,https://www.cnnbrasil.com.br/business/2020/02/...,Ibovespa tem nova queda com mercado ainda preo...,Preocupações com a propagação do novo coronaví...,CNN,preocupações propagação novo coronavírus poten...,ibovespa nova queda mercado ainda preocupado c...
1613,https://www.cnnbrasil.com.br/internacional/202...,Japonesa testa positivo pela segunda vez para ...,TÓQUIO - Uma guia de ônibus turístico no Japão...,CNN,tóquio guia ônibus turístico japão apresentou ...,japonesa testa positivo segunda vez coronavírus
1614,https://www.cnnbrasil.com.br/saude/2020/02/26/...,Primeiro brasileiro com coronavírus tem sintom...,O primeiro brasileiro com diagnóstico confir...,CNN,primeiro brasileiro diagnóstico confirmado cor...,primeiro brasileiro coronavírus sintomas brand...


In [10]:
def get_vocab(texts):
    vocab = ' '.join([text for text in texts])
    return sorted(set(vocab.split(' ')))

processed_texts_vocab = get_vocab(dataset.processed_text.values)
processed_titles_vocab = get_vocab(dataset.processed_title.values)

### Embedding, convoluções e LSTM

Resolva novamente a segunda questão da 3ª lista usando pelo menos duas
arquiteturas de redes neurais que utilizem camadas *Embedding*, convolucionais e
*LSTM*. Compare com os resultados obtidos anteriormente nas lista 3 e 5.

#### Gerando as sequências

In [30]:
TOKENIZER_NUM_WORDS = 2000

tokenizer = Tokenizer(num_words=TOKENIZER_NUM_WORDS)
tokenizer.fit_on_texts(dataset.processed_text)

In [31]:
sequences = tokenizer.texts_to_sequences(dataset.processed_text)
sequences = pad_sequences(sequences)

In [37]:
INPUT_DIMENSION = TOKENIZER_NUM_WORDS + 1
INPUT_LENGTH = 500
CONV_FILTERS = 32
CONV_KERNEL_SIZE = 3
LSTM_UNITS = 100

lstm_model = Sequential([
    Embedding(input_dim=INPUT_DIMENSION, output_dim=300),
    Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL_SIZE),
    LSTM(units=LSTM_UNITS),
    Dense(new)
])

print(lstm_model.summary())

BATCH_SIZE = 64
VALIDATION_SPLIT = 0.25

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(x=sequences, 
               y=dataset.label.apply(lambda label: 0 if label == 'BBC' else 1),
               batch_size=BATCH_SIZE,
               validation_split=VALIDATION_SPLIT,
               epochs=20)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 300)         600300    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, None, 32)          28832     
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               53200     
Total params: 682,332
Trainable params: 682,332
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
19/19 [==============================] - 153s 8s/step - loss: 5.5284 - accuracy: 0.0289 - val_loss: 5.2693 - val_accuracy: 0.4158
Epoch 2/20
19/19 [==============================] - 150s 8s/step - loss: 2.3381 - accuracy: 0.1097 - val_loss: 2.1834 - val_accuracy: 0.0000e+00
Epoch 3/20
19/19 [==============================] - 153s 8s/step - loss: 1.8571 - accu

KeyboardInterrupt: 

# Representação Doc2Vec

Vamos treinar um modelo de representação _doc2vec_ para os nossos documentos. Dado o conjunto de textos (lista `paragraphs`) vamos gerar a representação vetorial `doc_embedding`:

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
paragraphs = dataset.processed_text.values
paragraphs = [paragraph.split(' ') for paragraph in paragraphs]

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(paragraphs)]

In [ ]:
doc_embedding = Doc2Vec(documents, vector_size=300, window=10, min_count=2, workers=4)

In [ ]:
X = numpy.array([doc_embedding[i] for i, text in enumerate(dataset.processed_text.values)])

# Classificadores

Balanceando o dataset para conter 350 artigos _BBC_ e 350 artigos _CNN_ (700 artigos no total):

In [ ]:
X = X[:2*bbc.shape[0]]

In [ ]:
label2index = {
    'BBC': 0, 'CNN': 1
}
y = [label2index[l] for l in dataset.label.values[:700]]

Separando o conjunto de treinamento e teste:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

Treinando os classificadores:

### Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

Treinamento (TF-IFD):

In [ ]:
classifier = LogisticRegression().fit(X_train, y_train)

Resultado da predição (TF-IDF):

In [ ]:
y_pred = classifier.predict(X_test)

Matriz de confusão (TF-IDF):

In [ ]:
from matplotlib import pyplot
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(classifier, X_test, y_test)
pyplot.show()

Métricas de validação (TF-IDF):

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
log_metrics = {
    'Modelo': 'Regressão Logísitca + TF-IDF',
    'Acurácia': accuracy_score(y_test, y_pred),
    'Precisão': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred)
}
log_metrics

### Naive Bayes

### Support Vector Machines (SVM)

Treinamento (TF-IFD):

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier = SVC().fit(X_train, y_train)

Resultado da predição (TF-IDF):

In [ ]:
y_pred = classifier.predict(X_test)

Matriz de confusão (TF-IDF):

In [ ]:
plot_confusion_matrix(classifier, X_test, y_test)
pyplot.show()

Métricas de validação (TF-IDF):

In [ ]:
svm_metrics = {
    'Modelo': 'SVM + TF-IDF',
    'Acurácia': accuracy_score(y_test, y_pred),
    'Precisão': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred)
}
svm_metrics

Comparando os classificadores:

In [ ]:
pandas.DataFrame([log_metrics, svm_metrics])

# Agrupamento

Realizar um agrupamento dos dados através das seguintes etapas:

* Aplicar o algotimo __PCA__ preservando _95%_ da variância nos dados
* Aplicar o algoritmo ___k_-means__ nos dados projetados, utilizando o método _elbow_ para encontrar o valor de _k_ ótimo

Vamos realizar _stemming_ no texto pré-processado e vetorizar o resultado:

In [ ]:
X_proj = PCA(n_components=.95).fit_transform(X)

Dimensão dos pontos projetados: 

In [ ]:
X_proj.shape[1]

Plotando o gráfico da distorção:

In [ ]:
visualizer = KElbowVisualizer(KMeans(), k=(15, 30), metric='distortion')
visualizer.fit(X_proj)
visualizer.show() 

Observando o gráfico acima escolhemos $k=20$

In [ ]:
k = 20
kmeans = KMeans(n_clusters=k).fit(X_proj)

Projetando os pontos no espaço de visualização via __TSNE__:

In [ ]:
start_time = time.time()
X_tsne = TSNE(n_components=2).fit_transform(X_proj)
interval_time = time.time() - start_time

Tempo de execução:

In [ ]:
print('%s segundos' % interval_time)

Gráfico de dispersão dos pontos:

In [ ]:
color_palette = Category20[k]

In [ ]:
def scatter_plot(X, labels):
    scatter_plot = figure(plot_width=1000, plot_height=500)
    scatter_plot.circle(X[:, 0], X[:, 1], size=10, line_color=[color_palette[l] for l in labels], fill_color=[color_palette[l] for l in labels], fill_alpha=.8)
    show(scatter_plot)

scatter_plot(X_tsne, kmeans.labels_)

Projetando os pontos no espaço de visualização via __UMAP__:

In [ ]:
start_time = time.time()
X_umap = umap.UMAP().fit_transform(X_proj)
interval_time = time.time() - start_time

Tempo de execução:

In [ ]:
print('%s segundos' % interval_time)

In [ ]:
scatter_plot(X_umap, kmeans.labels_)